In [ ]:
import random
import re
from datetime import datetime

year = str(datetime.today().year)
month = str(datetime.today().month)
day = str(datetime.today().day)
date = year + '-' + month + '-' + day

USER = {'name': 'user', 'bank_name': [], 'bank_num': [], 'bank_balance': [], 'NP_points': 0, 'NP_balance': 0}
saved_recipients = {}
min_charge = 0
log_whole = ''
log_for_menu = ''


def get_user_input(question:str, *options, **options_with_return):

    '''
    Deliver Options to answer the question passing them as additional arguments or keyword-arguments.
    Pass arguments as a string. The function will return the string argument.
    If you want the return value be something else than a string, use keyword arguments to pass your option,
    it will return the value corresponding to the key.
    '''
    while True:
    #Ask Question with options
        print(f"{question}")
        for option in options:
            print(f"{option}")
        for option in options_with_return.keys():
            print(f"{option}")
        user_input = input('')
    #Return the Value if Input was correct
        if user_input in options:
            return user_input
        elif user_input in options_with_return.keys():
            return options_with_return[user_input]
    #Ask for new input
        else:
            print("Wrong Input. Please use one of the given options.")


def home():
    while True:
        print("\n--Home Menu--")
        print("If you want to go back To Home menu, enter '00' when you have to enter something.\nIf you want to Quit, enter'000'when you have to enter something.")
        menu = input(f"Hi {USER['name']}. Choose the menu. \n1. Check My NaverPay \n2. Top up\n3. Wire\n4. Make a payment\n5.Register a Giftcard\n6. My Activity\n7. coin toss event\n0. Quit\n===> ")
        if menu == '1':
            check_my_np()
        elif menu == '2':
            top_up_page()
        elif menu == '3':
            wire_page()
        elif menu == '4':
            payment_page()
        elif menu == '5':
            register_gift_card()
        elif menu == '6':
            print(log_for_menu)
        elif menu == '7':
            coin_toss_game()
        elif menu == '0':
            print("Thank you for Using Naver Pay")
            break
        else:
            print("Wrong input.\n")




def check_my_np():
    while True:
        print("\n--Check My NaverPay--")
        print(f"{USER['name']} have {USER['NP_balance']} KRW, {len(USER['bank_name'])} registered bank account, {USER['NP_points']} points.\n")
        if USER['NP_balance'] == 0 or len(USER['bank_name']) == 0:
            sub_menu = input("Choose the menu.\n1. Top up\n2. Register a bank account\n===> ")
            if sub_menu == '1':
                top_up_page()
            elif sub_menu == '2':
                new_acc()

            elif sub_menu in ['00', '000']:
                additional_menu(sub_menu)
            else:
                print("Wrong input.\n")
        return





def top_up_page():
    while True:
        print("\n--Top up--")
        choice_topup = input('To see charge activities or To proceed top up [1 = activities, 2 = top up] ===>')
        if choice_topup == '1':
            log_list = log_whole.split("\n")
            choice_activity = [i for i in log_list if "top_up" in i]
            if not choice_activity:
                print("no activity yet")
                return
            else:
                activity = '\n'.join(choice_activity)
                print(activity)
                return


        elif choice_topup == '2':
            if len(USER['bank_name']) == 0:
                print("No registered account. Please Register a bank account first.\n")
                new_acc()
                continue 
            else:
                check_acc()
                return




def check_acc():   # 계좌 조회
    while True:
        for i in range(0, len(USER['bank_balance'])):
            print(f"{i + 1}-> {USER['bank_name'][i]}, {USER['bank_num'][i]}, '계좌 잔액:' {USER['bank_balance'][i]}원")
        sub_menu = input("1. Register new bank account\n2. Use original bank account\n===> ")
        if sub_menu == '1':
            new_acc()
            return
        elif sub_menu == '2':
            original_acc()
            return
        elif sub_menu in ['00', '000']:
            additional_menu(sub_menu)
            return
        else:
            print("Wrong input.\n")




def new_acc():      # 새 계좌 등록
    while True:
        print("\n--Register a bank account--")
        acc_name = input("Bank name ex) Woori\n===> ")
        if acc_name.isalpha():
            acc_num = input("Bank account number without '-' ex) 1234567890\n===> ")
            if acc_num.isdigit():
                if acc_num in USER['bank_num']:
                    sub_menu = str(get_user_input("This bank account already registered. Do you want to register a new one? ", yes=1, no=0))
                    if sub_menu == '1':
                        new_acc()
                        return
                    elif sub_menu == '0':
                        check_acc()
                        return
                    elif sub_menu in ['00','000']:
                        additional_menu(sub_menu)
                        return
                    else:
                        print("Wrong input.\n")
                        new_acc()
                        return
                else:
                    acc_bal = input("How much is the bank account balance? Only number.\n===> ")
                    if acc_bal.isnumeric():
                        USER['bank_name'].append(acc_name)
                        USER['bank_num'].append(acc_num)
                        USER['bank_balance'].append(int(acc_bal))
                        my_log(1, None, NaverPay_balance=USER['NP_balance'], NaverPay_point=USER['NP_points'])
                        print("Register completed")
                        return
                    else:
                        print("Wrong input")
            else:
                print("Wrong input")
        else:
            print("Wrong input")


def original_acc():    # 기존 계좌 사용
    while True:
        for i in range(0, len(USER['bank_balance'])):
            print(f"{i + 1}-> {USER['bank_name'][i]}, {USER['bank_num'][i]}, ' balance:' {USER['bank_balance'][i]}KRW")
        acc_select = input("Which account do you want to withdraw? Just enter the number\n===> ")
        if acc_select.isdigit():
            acc_select = int(acc_select) - 1
            if acc_select in range(0, len(USER['bank_num'])):
                pass
            else:
                print("Wrong input")
                return
            sub_menu = str(get_user_input(f"{USER['bank_name'][acc_select]} {USER['bank_num'][acc_select]} balance: {USER['bank_balance'][acc_select]}KRW\nIs this {USER['name']}'s account?", yes=1, no=2))
            if sub_menu == '1':
                top_up(USER['bank_name'][acc_select], USER['bank_num'][acc_select], USER['bank_balance'][acc_select], acc_select)
                return
            elif sub_menu == '2':
                print("Choose another account")
                original_acc()
            elif sub_menu in ['00', '000']:
                additional_menu(sub_menu)
                return
            else:
                print("Wrong input.")
                original_acc()
        else:
            print("wrong input")




def top_up(bankname, banknum, bankbalance, acc_select):
    while True:
        #****
        if min_charge > 0:
            charge_page_money = str(min_charge)

        else:
            charge_page_money = input("How much do you want to top up?\n===> ")
            if charge_page_money.isnumeric():
                if int(charge_page_money) <= bankbalance:
                    USER['NP_balance'] += int(charge_page_money)
                    USER['bank_balance'][acc_select] -= int(charge_page_money)
                    print("charge completed")
                    comments = str(get_user_input('Do you want to leave comments?', yes=1, no=2)) #fixed
                    if comments == '1':
                        comments = input('write your comments >>')
                        my_log(2, comments, NaverPay_balance=USER['NP_balance'], NaverPay_point=USER['NP_points'])
                        return
                    elif comments =='2':
                        my_log(2, None, NaverPay_balance=USER['NP_balance'], NaverPay_point=USER['NP_points'])
                        return
                    else:
                        print("Wrong input.")
                        continue #continue? pass? return?
                    print("Naver Pay top up completed.")
                    print(f"{USER['name']} has {USER['NP_balance']}KRW.\nWithdraw account {bankname} {banknum} balance: {bankbalance}KRW.")
                    return
                else:
                    print("Bank account balance insufficient")
                    return
            else:
                print("Wrong input.")



def wire_page():
    while True:
        print("--Wire--")
        choice_wire = input('To see wire activities or To proceed wire [1 = activities, 2 = wire] ===>')
        if choice_wire == '1':
            log_list = log_whole.split("\n")
            choice_activity = [i for i in log_list if "send money" in i]
            if not choice_activity:
                print("no activity yet")
                return
            else:
                activity = '\n'.join(choice_activity)
                print(activity)
                return
        elif choice_wire == '2':
            wire_process()
            return
        elif choice_wire in ['00', '000']:
            additional_menu(choice_wire)
        else:
            print("Wrong input.")




def wire_process():
    w_amount = input("How much do you want to wire? ")
    if w_amount.isdigit():
        w_amount = int(w_amount)
        if w_amount <= USER['NP_balance']:
            USER['NP_balance'] -= w_amount
            w_acc_owner = input("Enter the account owner: ")
            w_acc_bank = input("Enter the bank name: ")
            w_acc_num = input("Enter the bank account number: ")
            nname(w_acc_owner, w_acc_bank, w_acc_num)
            return
        else:
            insuf = str(get_user_input("Naver Pay balance not enough. Want to top up?", yes=1,no=2))
            if insuf =='1':
                insufficient(w_amount, 1)
            elif insuf =='2':
                home()
                return
            elif insuf in ['00','000']:
                additional_menu(insuf)
    else:
        print("Wrong input")




def insufficient(w_amount, select):
    top_up_page()
    #wire 버전
    if select == 1:
        if USER['NP_balance'] >= w_amount:
            print(f"Wiring {w_amount} KRW completed.")
            USER['NP_balance'] -= w_amount
            comments = str(get_user_input('Do you want to leave comments?', yes=1, no=2))
            if comments == '1':
                comments = input('write your comments >>')
                my_log(5, comments, NaverPay_balance=USER['NP_balance'], NaverPay_point=USER['NP_points'])
            elif comments =='2':
                my_log(5, None, NaverPay_balance=USER['NP_balance'], NaverPay_point=USER['NP_points'])
            else:
                print("Wrong input.")
        else:
            print("Naver Pay balance still insufficient after top-up.")
    #pay버젼
    elif select == 2:
        if USER['NP_balance'] >= w_amount:
            print(f"paying {w_amount} KRW completed.")
            USER['NP_balance'] -= w_amount
            comments = str(get_user_input('Do you want to leave comments?', yes=1, no=2))
            if comments == '1':
                comments = input('write your comments >>')
                my_log(3, comments, NaverPay_balance=USER['NP_balance'], NaverPay_point=USER['NP_points'])
            elif comments =='2':
                my_log(3, None, NaverPay_balance=USER['NP_balance'], NaverPay_point=USER['NP_points'])
            else:
                print("Wrong input.")
        else:
            print("Naver Pay balance still insufficient after top-up.")





def nname(w_acc_owner, w_acc_bank, w_acc_num):  # 계좌 별명 저장
    while True:
        sub_menu = str(get_user_input(f"Do you want to make a nickname for {w_acc_owner}'s {w_acc_bank} {w_acc_num} and save?",yes=1,no=2))
        if sub_menu == '1':
            nickname = input("Please enter the nickname ex) Mom, Landlord: ")
            saved_recipients.update({nickname: [w_acc_owner, w_acc_bank, w_acc_num]})
            print("wire successfully done")
            comments = str(get_user_input('Do you want to leave comments?', yes=1, no=2))
            if comments == '1':
                comments = input('write your comments >>')
                my_log(5, comments, NaverPay_balance=USER['NP_balance'], NaverPay_point=USER['NP_points'])
            elif comments =='2':
                my_log(5, None, NaverPay_balance=USER['NP_balance'], NaverPay_point=USER['NP_points'])
            else:
                print("Wrong input.")
            return
        elif sub_menu == '2':
            return
        elif sub_menu in ['00', '000']:
            additional_menu(sub_menu)
        else:
            print("Wrong input.\n")






def payment_page():  # 결제 금액 정하기
    while True:
        print("\n--Make a payment--")
        choice_payment = input('To see payment activities or To proceed payment [1 = activities, 2 = payment] ===>')
        if choice_payment == '1':
            log_list = log_whole.split("\n")
            choice_activity = [i for i in log_list if "pay" in i]
            if not choice_activity:
                print("no activity yet")
                return
            else:
                activity = '\n'.join(choice_activity)
                print(activity)
                return
        elif choice_payment == '2':
            pay_amount = input("How much would you like to pay by Naver Pay? ")
            if pay_amount.isnumeric():
                pay_amount = int(pay_amount)
                points = pay_amount * 0.05
                print(f"You'll pay {pay_amount}KRW, and estimated points to get is {USER['NP_points']} points.")
                if pay_amount <= USER['NP_balance']:
                    payment(pay_amount,points)
                    return
                else:
                    print('insufficient balance')
                    insufficient(pay_amount,2)
                    return
            else:
                print("Wrong input.")
        elif choice_payment in ['00', '000']:
            additional_menu(choice_payment)
        else:
            print("wrong input")
            return





def payment(pay_amount, points):     
    while True:
        sub_menu = str(get_user_input(f"Do you want to pay {pay_amount}KRW by Naver Pay?",yes=1,no=2))
        if sub_menu == '1':
            USER['NP_balance'] -= pay_amount
            USER['NP_points'] += points
            comments = str(get_user_input('Do you want to leave comments?',yes=1,no=2)) #fixed
            if comments == '1':
                comments = input('write your comments >>')
                my_log(3, comments, NaverPay_balance=USER['NP_balance'], NaverPay_point=USER['NP_points'])
                return
            elif comments =='2':
                my_log(3, None, NaverPay_balance=USER['NP_balance'], NaverPay_point=USER['NP_points'])
                return
            else:
                print("Wrong input.")
                continue 
            print(f"Payment completed.\n{USER['name']} paid {pay_amount}KRW, got new {points} points.\nNaver Pay points: {USER['NP_points']} points, Naver Pay balance: {USER['NP_balance']} KRW.")
            roulette()
            return
        elif sub_menu == '2':
            print("Payment cancelled")
            return
        elif sub_menu in ['00', '000']:
            additional_menu(sub_menu)
        else:
            print("Wrong input\n")


def register_gift_card():
        gift_code_regex = re.compile(r'([a-zA-Z])-(\d{4}-\d{4}-\d{3})')
        while True:
            gift_code = input('Enter the giftcard-code with the Pattern: X-0000-0000-000: ')
            gift_card = gift_code_regex.match(gift_code)
            if gift_card is not None:
                break
            else:
                print('The code was incorrect')

        amount_char, code = gift_card.groups()
        switcher = {'p': 50000, 'q': 30000, 'r': 10000}
        amount = switcher.get(amount_char, 5000)
        while True:
            confirmation = str(get_user_input(f'Please confirm charge of {amount} on to your Naver account', yes=1,no=2))
            if confirmation == '1':
                USER['NP_balance']+=amount
                print('Balance has been updated to {}'.format(USER['NP_balance']))
                my_log(4, None, NaverPay_balance=USER['NP_balance'], NaverPay_point=USER['NP_points'])
                break
            elif confirmation == '2':
                print('Registering canceled')
                return
            elif confirmation in ['00', '000']:
                additional_menu(confirmation)
            else:
                print('Please enter a valid input')




def my_log(choice, comments, **kwargs):
    global log_whole, log_for_menu
    kind = ["register account","top_up","pay","register gift card","send money" ]
    kinds = {i:kind for (i,kind) in enumerate(kind,1)}
    log = f'{date:>15} | {kinds[choice]:>10} | {kwargs} |' +'\n'
    if choice in kinds.keys():
        log_for_menu += log
        def log_comments():
            global log_whole
            nonlocal log
            log = f'{date:>15} | {kinds[choice]:>10} | {kwargs} | {comments}' +'\n'
            log_whole += log
            return print(log)
    else:
        print("wrong input")

    return log_comments()




def roulette():
    roulette_list = [10, 20, 50, 100, 500, 1000, 5000]
    numbers = random.choice(roulette_list)
    print(f"congratulation! You got {numbers} points for bonus!")
    USER['NP_points'] += numbers
    print(f"You have {USER['NP_points']} points now.")



def additional_menu(zeros):
    if zeros == '00':
        print("Go back to Home Menu")
        home()
    elif zeros == '000':
        home(menu = '0')


def coin_toss_game():
    #Helper Functions
    enough_points = lambda points, bet: True if points >= bet else False
    multiply_points = lambda points, multiplier: points * multiplier

    def flip_coin():
        flip = random.randint(0, 1)
        if flip == 0:
            print("The coin shows: Heads")
            return 0
        else:
            print("The coin shows: Tails")
            return 1

    print("Welcome to the Toin Coss Game. A coin will be tossed and you can bet on the side.",
          "\nIf you guessed right your bet will be doubled! If you're wrong you'll lose the points.")

    while True: #Game Loop
        if enough_points(USER["NP_points"], 1):
            try:
                print(f"You have {USER['NP_points']} at the moment")
                bet = int(input("How many Points would you like to bet?"))
                if enough_points(int(USER["NP_points"]), bet):
                    USER['NP_points'] -= bet
                    guess = get_user_input("Which Side will the coin show?", heads=0, tails=1)
                    #FLIP COIN
                    if flip_coin() == guess:
                        points_won = multiply_points(bet, 2)
                        USER['NP_points'] += points_won
                        print(f"You guessed it! You win {points_won} Points!")
                    else:
                        print(f"You guessed wrong! You lose {bet} Points")

                    if not get_user_input("Another Round?", yes=True, no=False):
                        break

                else:
                    print("Not enough points.")

            except:
                print("Wrong input. Please enter a number!")

        else:
            print("You have no Points left. Returning to menu...")
            break




user = str(input("Hi. It's Naver Pay. Please enter your name ===> "))
USER['name'] = user
home()



